In [ ]:
import numpy as np
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import PersistenceEntropy
from gtda.plotting import plot_diagram

# Generate a 2D point cloud arranged in an annulus (circle with a hole)
np.random.seed(42)
theta = np.linspace(0, 2 * np.pi, 100)
r_inner = 0.5
r_outer = 1.0
r = np.random.uniform(r_inner, r_outer, 100)
x = r * np.cos(theta)
y = r * np.sin(theta)
point_cloud = np.column_stack((x, y))

# Compute persistent homology
vr = VietorisRipsPersistence(homology_dimensions=[0, 1])
diagrams = vr.fit_transform([point_cloud])

# Plot the persistence diagram to confirm the H1 loop
plot_diagram(diagrams[0])

In [ ]:
import torch
from torch.optim import Adam

# Convert point cloud to a trainable PyTorch tensor
point_cloud_torch = torch.tensor(point_cloud, dtype=torch.float32, requires_grad=True)

# Define differentiable topological loss
def topological_loss(diagram, dimension=1, target_persistence=0.0):
    # Extract the H1 features
    h1_features = diagram[0][diagram[0][:, 2] == dimension]
    if len(h1_features) > 0:
        # Compute persistence (death - birth) of the most prominent H1 feature
        persistences = h1_features[:, 1] - h1_features[:, 0]
        max_persistence = torch.max(torch.tensor(persistences))
        return (max_persistence - target_persistence) ** 2
    else:
        return torch.tensor(0.0)

# Optimization loop
optimizer = Adam([point_cloud_torch], lr=0.01)
for epoch in range(100):
    optimizer.zero_grad()
    
    # Recompute persistent homology (note: giotto-tda is not natively differentiable, this is a proxy; in practice, use differentiable TDA layers if available)
    diagrams = vr.fit_transform([point_cloud_torch.detach().numpy()])  # Detach for non-diff computation
    loss = topological_loss(diagrams)
    loss.backward()  # This assumes a differentiable proxy; adjust as needed
    optimizer.step()
    if epoch % 10 == 0:
        print(f'Epoch {epoch}: Loss = {loss.item()}')

In [ ]:
import matplotlib.pyplot as plt

# Compute final diagrams after optimization
final_point_cloud = point_cloud_torch.detach().numpy()
final_diagrams = vr.fit_transform([final_point_cloud])

# Create side-by-side visualization
fig, axs = plt.subplots(2, 2, figsize=(12, 8))

# Initial point cloud
axs[0, 0].scatter(point_cloud[:, 0], point_cloud[:, 1])
axs[0, 0].set_title('Initial Point Cloud (Annulus)')

# Initial persistence diagram
axs[0, 1].scatter(diagrams[0][:, 0], diagrams[0][:, 1], c=diagrams[0][:, 2])
axs[0, 1].set_title('Initial Persistence Diagram')

# Final point cloud
axs[1, 0].scatter(final_point_cloud[:, 0], final_point_cloud[:, 1])
axs[1, 0].set_title('Optimized Point Cloud')

# Final persistence diagram
axs[1, 1].scatter(final_diagrams[0][:, 0], final_diagrams[0][:, 1], c=final_diagrams[0][:, 2])
axs[1, 1].set_title('Final Persistence Diagram')

plt.tight_layout()
plt.show()